In [1]:
import pandas as pd

In [2]:
pizza = pd.read_csv('pizza_sales.csv')

In [3]:
pizza.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01-01-2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01-01-2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01-01-2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,01-01-2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,01-01-2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [4]:
pizza['order_date'] = (
    pizza['order_date']
    .str.replace('\n', '', regex=False)
    .pipe(pd.to_datetime, format='%d-%m-%Y')
)

In [5]:
pizza[pizza['order_date'].isna()]

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name


In [6]:
pizza['order_date'].head()
pizza['order_date'].dtype

dtype('<M8[ns]')

In [7]:
total_revenue = pizza['total_price'].sum()

In [8]:
avg_order_value = pizza['total_price'].sum() / pizza['order_id'].nunique()

In [9]:
total_pizzas_sold = pizza['quantity'].sum()

In [10]:
total_orders = pizza['order_id'].nunique()

In [11]:
avg_pizzas_per_order = round(
    pizza['quantity'].sum() / pizza['order_id'].nunique(), 2
)

In [12]:
from IPython.display import display

kpi_df = pd.DataFrame({
    "Metric": [
        "Total Revenue",
        "Average Order Value",
        "Total Pizzas Sold",
        "Total Orders",
        "Average Pizzas per Order"
    ],
    "Value": [
        total_revenue,
        avg_order_value,
        total_pizzas_sold,
        total_orders,
        avg_pizzas_per_order
    ]
})

In [13]:
display(kpi_df)

,Metric,Value
0,Total Revenue,817860.050000
1,Average Order Value,38.307262
2,Total Pizzas Sold,49574.000000
3,Total Orders,21350.000000
4,Average Pizzas per Order,2.320000


In [14]:
daily_trend = (
    pizza
    .assign(order_day=pizza['order_date'].dt.day_name())
    .groupby('order_day')['order_id']
    .nunique()
    .reset_index(name='total_orders')
)

In [15]:
monthly_trend = (
    pizza
    .assign(month_name=pizza['order_date'].dt.month_name())
    .groupby('month_name')['order_id']
    .nunique()
    .reset_index(name='total_orders')
)

In [16]:
display(daily_trend)
display(monthly_trend)

,order_day,total_orders
0,Friday,3538
1,Monday,2794
2,Saturday,3158
3,Sunday,2624
4,Thursday,3239
5,Tuesday,2973
6,Wednesday,3024


,month_name,total_orders
0,April,1799
1,August,1841
2,December,1680
3,February,1685
4,January,1845
5,July,1935
6,June,1773
7,March,1840
8,May,1853
9,November,1792


In [17]:
category_sales = (
    pizza
    .groupby('pizza_category', as_index=False)['total_price']
    .sum()
)

category_sales['pct_sales'] = round(
    category_sales['total_price'] * 100 / pizza['total_price'].sum(), 2
)

In [18]:
size_sales = (
    pizza
    .groupby('pizza_size', as_index=False)['total_price']
    .sum()
    .sort_values('pizza_size')
)

size_sales['pct_sales'] = round(
    size_sales['total_price'] * 100 / pizza['total_price'].sum(), 2
)

In [19]:
feb_category_sales = (
    pizza[pizza['order_date'].dt.month == 2]
    .groupby('pizza_category', as_index=False)['quantity']
    .sum()
    .sort_values('quantity', ascending=False)
)

In [20]:
display(size_sales)
display(category_sales)
display(feb_category_sales)

,pizza_size,total_price,pct_sales
0,L,375318.70,45.89
1,M,249382.25,30.49
2,S,178076.50,21.77
3,XL,14076.00,1.72
4,XXL,1006.60,0.12


,pizza_category,total_price,pct_sales
0,Chicken,195919.50,23.96
1,Classic,220053.10,26.91
2,Supreme,208197.00,25.46
3,Veggie,193690.45,23.68


,pizza_category,quantity
1,Classic,1178
2,Supreme,964
3,Veggie,944
0,Chicken,875


In [21]:
top5_revenue = (
    pizza
    .groupby('pizza_name', as_index=False)['total_price']
    .sum()
    .sort_values('total_price', ascending=False)
    .head(5)
)

In [22]:
bottom5_revenue = (
    pizza
    .groupby('pizza_name', as_index=False)['total_price']
    .sum()
    .sort_values('total_price')
    .head(5)
)

In [23]:
display(top5_revenue)
display(bottom5_revenue)

,pizza_name,total_price
30,The Thai Chicken Pizza,43434.25
0,The Barbecue Chicken Pizza,42768.00
4,The California Chicken Pizza,41409.50
7,The Classic Deluxe Pizza,38180.50
26,The Spicy Italian Pizza,34831.25


,pizza_name,total_price
2,The Brie Carre Pizza,11588.50
11,The Green Garden Pizza,13955.75
28,The Spinach Supreme Pizza,15277.75
16,The Mediterranean Pizza,15360.50
27,The Spinach Pesto Pizza,15596.00


In [24]:
top5_quantity = (
    pizza
    .groupby('pizza_name', as_index=False)['quantity']
    .sum()
    .sort_values('quantity', ascending=False)
    .head(5)
)

In [25]:
bottom5_quantity = (
    pizza
    .groupby('pizza_name', as_index=False)['quantity']
    .sum()
    .sort_values('quantity')
    .head(5)
)

In [26]:
display(top5_quantity)
display(bottom5_quantity)

,pizza_name,quantity
7,The Classic Deluxe Pizza,2453
0,The Barbecue Chicken Pizza,2432
12,The Hawaiian Pizza,2422
20,The Pepperoni Pizza,2418
30,The Thai Chicken Pizza,2371


,pizza_name,quantity
2,The Brie Carre Pizza,490
16,The Mediterranean Pizza,934
3,The Calabrese Pizza,937
28,The Spinach Supreme Pizza,950
24,The Soppressata Pizza,961


In [27]:
top5_orders = (
    pizza
    .groupby('pizza_name')['order_id']
    .nunique()
    .reset_index(name='total_orders')
    .sort_values('total_orders', ascending=False)
    .head(5)
)

In [28]:
bottom5_orders = (
    pizza
    .groupby('pizza_name')['order_id']
    .nunique()
    .reset_index(name='total_orders')
    .sort_values('total_orders')
    .head(5)
)

In [29]:
display(top5_orders)
display(bottom5_orders)

,pizza_name,total_orders
7,The Classic Deluxe Pizza,2329
12,The Hawaiian Pizza,2280
20,The Pepperoni Pizza,2278
0,The Barbecue Chicken Pizza,2273
30,The Thai Chicken Pizza,2225


,pizza_name,total_orders
2,The Brie Carre Pizza,480
16,The Mediterranean Pizza,912
28,The Spinach Supreme Pizza,918
3,The Calabrese Pizza,918
6,The Chicken Pesto Pizza,938
